In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from math import *
from numpy import *
from sklearn.pipeline import Pipeline
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
import os
import string
import gensim
from gensim.models import Word2Vec
import pandas as pd
import random
import matplotlib.pyplot as plt
import re
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from nltk.classify import NaiveBayesClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score,recall_score,precision_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn import svm
from sklearn.svm import LinearSVC

In [4]:
def striprtf(text):
   pattern = re.compile(r"\\([a-z]{1,32})(-?\d{1,10})?[ ]?|\\'([0-9a-f]{2})|\\([^a-z])|([{}])|[\r\n]+|(.)", re.I)
   # control words which specify a "destionation".
   destinations = frozenset((
      'aftncn','aftnsep','aftnsepc','annotation','atnauthor','atndate','atnicn','atnid',
      'atnparent','atnref','atntime','atrfend','atrfstart','author','background',
      'bkmkend','bkmkstart','blipuid','buptim','category','colorschememapping',
      'colortbl','comment','company','creatim','datafield','datastore','defchp','defpap',
      'do','doccomm','docvar','dptxbxtext','ebcend','ebcstart','factoidname','falt',
      'fchars','ffdeftext','ffentrymcr','ffexitmcr','ffformat','ffhelptext','ffl',
      'ffname','ffstattext','field','file','filetbl','fldinst','fldrslt','fldtype',
      'fname','fontemb','fontfile','fonttbl','footer','footerf','footerl','footerr',
      'footnote','formfield','ftncn','ftnsep','ftnsepc','g','generator','gridtbl',
      'header','headerf','headerl','headerr','hl','hlfr','hlinkbase','hlloc','hlsrc',
      'hsv','htmltag','info','keycode','keywords','latentstyles','lchars','levelnumbers',
      'leveltext','lfolevel','linkval','list','listlevel','listname','listoverride',
      'listoverridetable','listpicture','liststylename','listtable','listtext',
      'lsdlockedexcept','macc','maccPr','mailmerge','maln','malnScr','manager','margPr',
      'mbar','mbarPr','mbaseJc','mbegChr','mborderBox','mborderBoxPr','mbox','mboxPr',
      'mchr','mcount','mctrlPr','md','mdeg','mdegHide','mden','mdiff','mdPr','me',
      'mendChr','meqArr','meqArrPr','mf','mfName','mfPr','mfunc','mfuncPr','mgroupChr',
      'mgroupChrPr','mgrow','mhideBot','mhideLeft','mhideRight','mhideTop','mhtmltag',
      'mlim','mlimloc','mlimlow','mlimlowPr','mlimupp','mlimuppPr','mm','mmaddfieldname',
      'mmath','mmathPict','mmathPr','mmaxdist','mmc','mmcJc','mmconnectstr',
      'mmconnectstrdata','mmcPr','mmcs','mmdatasource','mmheadersource','mmmailsubject',
      'mmodso','mmodsofilter','mmodsofldmpdata','mmodsomappedname','mmodsoname',
      'mmodsorecipdata','mmodsosort','mmodsosrc','mmodsotable','mmodsoudl',
      'mmodsoudldata','mmodsouniquetag','mmPr','mmquery','mmr','mnary','mnaryPr',
      'mnoBreak','mnum','mobjDist','moMath','moMathPara','moMathParaPr','mopEmu',
      'mphant','mphantPr','mplcHide','mpos','mr','mrad','mradPr','mrPr','msepChr',
      'mshow','mshp','msPre','msPrePr','msSub','msSubPr','msSubSup','msSubSupPr','msSup',
      'msSupPr','mstrikeBLTR','mstrikeH','mstrikeTLBR','mstrikeV','msub','msubHide',
      'msup','msupHide','mtransp','mtype','mvertJc','mvfmf','mvfml','mvtof','mvtol',
      'mzeroAsc','mzeroDesc','mzeroWid','nesttableprops','nextfile','nonesttables',
      'objalias','objclass','objdata','object','objname','objsect','objtime','oldcprops',
      'oldpprops','oldsprops','oldtprops','oleclsid','operator','panose','password',
      'passwordhash','pgp','pgptbl','picprop','pict','pn','pnseclvl','pntext','pntxta',
      'pntxtb','printim','private','propname','protend','protstart','protusertbl','pxe',
      'result','revtbl','revtim','rsidtbl','rxe','shp','shpgrp','shpinst',
      'shppict','shprslt','shptxt','sn','sp','staticval','stylesheet','subject','sv',
      'svb','tc','template','themedata','title','txe','ud','upr','userprops',
      'wgrffmtfilter','windowcaption','writereservation','writereservhash','xe','xform',
      'xmlattrname','xmlattrvalue','xmlclose','xmlname','xmlnstbl',
      'xmlopen',
   ))
   # Translation of some special characters.
   specialchars = {
      'par': '\n',
      'sect': '\n\n',
      'page': '\n\n',
      'line': '\n',
      'tab': '\t',
      'emdash': u'\u2014',
      'endash': u'\u2013',
      'emspace': u'\u2003',
      'enspace': u'\u2002',
      'qmspace': u'\u2005',
      'bullet': u'\u2022',
      'lquote': u'\u2018',
      'rquote': u'\u2019',
      'ldblquote': u'\201C',
      'rdblquote': u'\u201D', 
   }
   stack = []
   ignorable = False       # Whether this group (and all inside it) are "ignorable".
   ucskip = 1              # Number of ASCII characters to skip after a unicode character.
   curskip = 0             # Number of ASCII characters left to skip
   out = []                # Output buffer.
   for match in pattern.finditer(text):
      word,arg,hex,char,brace,tchar = match.groups()
      if brace:
         curskip = 0
         if brace == '{':
            # Push state
            stack.append((ucskip,ignorable))
         elif brace == '}':
            # Pop state
            ucskip,ignorable = stack.pop()
      elif char: # \x (not a letter)
         curskip = 0
         if char == '~':
            if not ignorable:
                out.append(u'\xA0')
         elif char in '{}\\':
            if not ignorable:
               out.append(char)
         elif char == '*':
            ignorable = True
      elif word: # \foo
         curskip = 0
         if word in destinations:
            ignorable = True
         elif ignorable:
            pass
         elif word in specialchars:
            out.append(specialchars[word])
         elif word == 'uc':
            ucskip = int(arg)
         elif word == 'u':
            c = int(arg)
            if c < 0: c += 0x10000
            if c > 127: out.append(chr(c))
            else: out.append(chr(c))
            curskip = ucskip
      elif hex: # \'xx
         if curskip > 0:
            curskip -= 1
         elif not ignorable:
            c = int(hex,16)
            if c > 127: out.append(chr(c))
            else: out.append(chr(c))
      elif tchar:
         if curskip > 0:
            curskip -= 1
         elif not ignorable:
            out.append(tchar)
   return ''.join(out)

In [5]:
data = pd.read_csv('./train_pe.csv')
df = pd.read_csv("train_pe.csv", usecols=[2])
for i,j in df.iterrows(): 
    text=df.Result[i]
    df.Result[i]=striprtf(text)
for i,j in df.iterrows(): 
    text=df.Result[i]
    df.Result[i]=text.lower()
to_remove = ['no', 'nor']
new_stopwords = set(stopwords.words('english')).difference(to_remove)
for i,j in df.iterrows():                  
    text=df.Result[i]
    words=text.split()
    meaningful_words = [w for w in words if not w in new_stopwords]   
    df.Result[i]=(" ".join( meaningful_words))
df

,Result
0,investigation: x-ray - chest (portable) result...
1,hrct chest (non contrast) spiral scanning thor...
2,chest x-ray pa view no significant interval ch...
3,investigation: x-ray - chest ap (portable) vie...
4,x-ray chest pa/ap view 26-aug-2017: results: r...
5,x-ray chest pa/ap view 28-aug-2017: results: r...
6,x-ray chest pa/ap view 22-aug-2017: results: c...
7,usg whole abdomen 22-aug-2017: liver: normal s...
8,x-ray chest pa/ap view 23-aug-2017: results: c...
9,x-ray chest pa/ap view 21-aug-2017: results: n...


In [6]:
workabledata = pd.concat([data, df], axis=1)
del data['Result']
workabledata = pd.concat([data, df], axis=1)
workabledata

,Identifier,TestName,Pleural_effusion,Result
0,2,X-Ray Chest PA/AP View,0,investigation: x-ray - chest (portable) result...
1,3,CT Thorax / Chest - HRCT Plain,0,hrct chest (non contrast) spiral scanning thor...
2,4,X-ray (any Portable single exposure),0,chest x-ray pa view no significant interval ch...
3,5,X-Ray Chest PA/AP View,0,investigation: x-ray - chest ap (portable) vie...
4,6,X-Ray Chest PA/AP View,0,x-ray chest pa/ap view 26-aug-2017: results: r...
5,7,X-Ray Chest PA/AP View,0,x-ray chest pa/ap view 28-aug-2017: results: r...
6,8,X-Ray Chest PA/AP View,0,x-ray chest pa/ap view 22-aug-2017: results: c...
7,9,USG Whole Abdomen,1,usg whole abdomen 22-aug-2017: liver: normal s...
8,10,X-Ray Chest PA/AP View,0,x-ray chest pa/ap view 23-aug-2017: results: c...
9,11,X-Ray Chest PA/AP View,0,x-ray chest pa/ap view 21-aug-2017: results: n...


In [7]:
test_related_pe=['CT Thorax / Chest - HRCT Contrast','CT Thorax / Chest - HRCT Plain','CT Thorax / Chest - Plain','CT Whole Abdomen - Plain','CT Whole Abdomen with Contrast','CT Whole Thorax (Contrast)','CT Whole Thorax (Plain)','MRI Cholangiography (MRCP)','Ultrasound Chest','Ultrasound KUB','Ultrasound Whole Abdomen','USG Chest','USG Upper Abdomen','USG Whole Abdomen','X-ray (any Portable single exposure)','X-Ray Chest PA/AP View']

In [8]:
import re
unique_words=set()
punctuations = list(string.punctuation)
for i,j in workabledata.iterrows():
    test=workabledata.TestName[i]
    text=workabledata.Result[i]
    if(test in test_related_pe ):
        punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~+/='''
        text3=""
        for char in text:
            if char not in punctuations:
                text3 = text3 + char
        unique_words_temp=set(text3.split())
        unique_words=unique_words.union(unique_words_temp)
        df.Result[i]=text3
workabledata = pd.concat([data, df], axis=1)
#text=workabledata.Result[503]
#punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~+/='''
#text3=""
#for char in text:
#    if char not in punctuations:
#        text3 = text3 + char
#text3
#len(unique_words)
unique_words_list=[]
for e in unique_words:
    #print(e)
    unique_words_list.append(e)
unique_words_list
def oopsie(x):
    flag=False
    for i in x:
        if(i.isdigit()):
            flag=True
            break
    return flag
li=[x for x in unique_words_list if not oopsie(x)]
sorted(li)
len(li)
for x in li:
    if x in new_stopwords:
        li.remove(x)
len(li)
li.remove('kk')
li
workabledata

,Identifier,TestName,Pleural_effusion,Result
0,2,X-Ray Chest PA/AP View,0,investigation xray chest portable results het...
1,3,CT Thorax / Chest - HRCT Plain,0,hrct chest non contrast spiral scanning thorax...
2,4,X-ray (any Portable single exposure),0,chest xray pa view no significant interval cha...
3,5,X-Ray Chest PA/AP View,0,investigation xray chest ap portable view res...
4,6,X-Ray Chest PA/AP View,0,xray chest paap view 26aug2017 results rotatio...
5,7,X-Ray Chest PA/AP View,0,xray chest paap view 28aug2017 results rotatio...
6,8,X-Ray Chest PA/AP View,0,xray chest paap view 22aug2017 results cv line...
7,9,USG Whole Abdomen,1,usg whole abdomen 22aug2017 liver normal size ...
8,10,X-Ray Chest PA/AP View,0,xray chest paap view 23aug2017 results cv line...
9,11,X-Ray Chest PA/AP View,0,xray chest paap view 21aug2017 results no foca...


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
new_stopwords = set(stopwords.words('english')).difference(to_remove)
vectorizer = CountVectorizer(lowercase=True, stop_words=new_stopwords)
matrix = vectorizer.fit_transform(workabledata.Result)
s=workabledata.copy()
sd=workabledata[workabledata.Pleural_effusion>0]
sdt=sd.copy()
ops=1
sd = sd.reset_index(drop=True)
sd
from sklearn.feature_extraction.text import CountVectorizer
new_stopwords = set(stopwords.words('english')).difference(to_remove)
vectorizer = CountVectorizer(lowercase=True, stop_words=new_stopwords)
matrix = vectorizer.fit_transform(s.Result)
matrix=matrix.toarray()
matrix
x=matrix
y=s.Pleural_effusion
yt=y.array
bagsofwords = [Counter(re.findall(r'\w+', txt))for txt in workabledata.Result]

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.25,random_state=123)
#algo1-Multinomial Naive Bayes
naive = MultinomialNB()
print("-------------------------------")
print("Multinomial Naive Bayes")
classifier = naive.fit(x_train,y_train)
predict = classifier.predict(x_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,predict)
print(cm)
print(accuracy_score(y_test,predict))
print(f1_score(y_test,predict))
print(precision_score(y_test,predict))
print(recall_score(y_test,predict))
#algo2-Bernoulli Naive Bayes
print("-------------------------------")
print("Bernoulli Naive Bayes")
BernNB = BernoulliNB(binarize = 0.095)
BernNB.fit(x_train,y_train)
y_pred_B = BernNB.predict(x_test)
cm = confusion_matrix(y_test,y_pred_B)
print(cm)
print (accuracy_score(y_test,y_pred_B))
print (f1_score(y_test,y_pred_B))
print(precision_score(y_test,y_pred_B))
print(recall_score(y_test,y_pred_B))
#algo3-Linear SVM
clf = LinearSVC()
clf.fit(x_train, y_train) 
y_pred_SVM=clf.predict(x_test)
print("-------------------------------")
print("Linear SVC")
cm = confusion_matrix(y_test,y_pred_SVM)
print(cm)
print (accuracy_score(y_test,y_pred_SVM))
print (f1_score(y_test,y_pred_SVM))
print(precision_score(y_test,y_pred_SVM))
print(recall_score(y_test,y_pred_SVM))
#algo3-Linear SVM
clf = svm.SVC()
clf.fit(x_train, y_train) 
y_pred_SVM2=clf.predict(x_test)
print("-------------------------------")
print("C-Support Vector Classification ")
cm = confusion_matrix(y_test,y_pred_SVM2)
print(cm)
print (accuracy_score(y_test,y_pred_SVM2))
print (f1_score(y_test,y_pred_SVM2))
print(precision_score(y_test,y_pred_SVM2))
print(recall_score(y_test,y_pred_SVM2))


-------------------------------
Multinomial Naive Bayes
[[308 109]
 [  8  81]]
0.7687747035573123
0.5806451612903226
0.4263157894736842
0.9101123595505618
-------------------------------
Bernoulli Naive Bayes
[[288 129]
 [ 25  64]]
0.6956521739130435
0.45390070921985815
0.3316062176165803
0.7191011235955056
-------------------------------
Linear SVC
[[414   3]
 [  1  88]]


/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.9920948616600791
0.9777777777777779
0.967032967032967
0.9887640449438202
-------------------------------
C-Support Vector Classification 
[[417   0]
 [ 85   4]]
0.8320158102766798
0.08602150537634408
1.0
0.0449438202247191
